In [222]:
import requests
import json
import re
import pandas as pd 
import numpy as np
import time

In [375]:
df = pd.read_csv('data/top_10000_ingredients.csv')

df = df.iloc[2185:]



In [358]:
api_key = "6QckmmkUv58RO4eu9eiXKUD1a0L60mSjTnHfa036"
def search_usda_food_with_fdcid(fdcid):
    '''
    given fdc id, return the detailed info
    '''
    url = "https://api.nal.usda.gov/fdc/v1/food/{}?api_key={}".format(fdcid, api_key)
    response = requests.request("GET", url)

    if response.status_code != 200:
        return ''
    
    return response.json()
def search_fdcid(ingredient):
    '''
    Return the fdc id of the search for the first one with food portions
    '''
    url = "https://api.nal.usda.gov/fdc/v1/foods/search?query={}&model=foodPortion&api_key=6QckmmkUv58RO4eu9eiXKUD1a0L60mSjTnHfa036"\
          .format(ingredient)
    response = requests.request("GET", url)

    if response.status_code != 200:
        return None
    
    for food in response.json().get("foods"):
        fdc_id = food.get("fdcId")
        json_data = search_usda_food_with_fdcid(fdc_id)
        if isinstance(json_data, str):
            continue
#         print(fdc_id)
#         print(search_usda_food_with_fdcid(fdc_id).get("foodPortions"))
        if( json_data.get("foodPortions") !=None ):
            if( len(json_data.get("foodPortions")) > 0 ):
                return fdc_id
    return None


In [ ]:
import time

ingredient_density={}
start = time.time()
ing_name=[]
standard_vol=[]
standard_unit=[]
standard_weight=[]
for index, row in df.iterrows():
    name = row.ingredients
#     print(name)
#     print(row.ingredient)
# for page in range(201,220):
#     time.sleep(5.0)

#     fdcid_list , names = search_fdcid(page)
#     for name, fdcid in zip(names, fdcid_list):
#         json_data = search_usda_food_with_fdcid(fdcid)
    fdcid = search_fdcid(row.ingredients)
    if fdcid == None:
        json_data={"foodPortions":[{}]}
    else:
        json_data = search_usda_food_with_fdcid(fdcid)

        if isinstance(json_data, str):
            continue
        elif 'portionDescription' in json_data.get("foodPortions")[0].keys():
            ing_amt = json_data.get("foodPortions")[0]['portionDescription'].split(' ')[0]
            if ing_amt.isdigit():
                standard_vol.append(int(ing_amt))
                standard_unit.append(json_data.get("foodPortions")[0]['portionDescription'].split(' ')[1])
                standard_weight.append(json_data.get("foodPortions")[0]['gramWeight'])
                ing_name.append(name)                  
        elif ('amount' in json_data.get("foodPortions")[0].keys()) and ('modifier' in json_data.get("foodPortions")[0].keys()):
            standard_vol.append(json_data.get("foodPortions")[0]["amount"])
            standard_unit.append(json_data.get("foodPortions")[0]["modifier"])
            standard_weight.append(json_data.get("foodPortions")[0]["gramWeight"])
            ing_name.append(name)                        
        else:
            standard_vol.append(np.nan)
            standard_weight.append(np.nan)
            standard_unit.append('')
            ing_name.append(name)                         

           
# ing_name = [item for sublist in ing_name for item in sublist]
ingredient_density['ingredient']= ing_name      
ingredient_density['standard_vol']=standard_vol
ingredient_density['standard_weight_gm'] = standard_weight
ingredient_density['standard_unit'] =standard_unit
ingredient_density_df = pd.DataFrame.from_dict(ingredient_density)

print("time in minutes {}".format((time.time() - start)/60))

In [360]:
ingredient_density['ingredient']= ing_name      
ingredient_density['standard_vol']=standard_vol
ingredient_density['standard_weight_gm'] = standard_weight
ingredient_density['standard_unit'] =standard_unit
ingredient_density_df = pd.DataFrame.from_dict(ingredient_density)

In [371]:
ingredient_density_df.tail(15)

,ingredient,standard_vol,standard_weight_gm,standard_unit
1546,lbs red potatoes,1.0,299.0,"potato large (3"" to 4-1/4"" dia."
1547,flax seed meal,1.0,168.0,cup
1548,hard-boiled egg,1.0,8.5,tbsp
1549,eggnog,1.0,256.0,cup
1550,rose water,1.0,30.0,fl
1551,fat-free evaporated milk,1.0,252.0,cup
1552,anise seed,1.0,6.7,"tbsp, whole"
1553,tablespoons softened butter,1.0,16.0,tablespoon
1554,whole carrot,1.0,120.0,cup
1555,skim lowfat milk,1.0,244.0,cup


In [362]:
# ing_dens_comb = ingredient_density_df
ing_dens_comb = pd.concat([ing_dens_comb,ingredient_density_df], ignore_index=True)

In [363]:
ing_dens_comb.shape

(2080, 4)

In [364]:
ing_dens_comb_copy = ing_dens_comb
ing_dens_comb_copy.shape

(2080, 4)

In [365]:
ing_dens_comb.tail()

,ingredient,standard_vol,standard_weight_gm,standard_unit
2075,crumbled goat cheese,1.0,140.0,"cup,"
2076,fresh minced garlic,1.0,21.0,"slice (4-1/4"" x 4-1/4"" x 1/16"")"
2077,cloves of garlic,1.0,2.1,tsp
2078,pint strawberry,1.0,150.0,cup
2079,tablespoons ginger,1.0,240.0,cup


In [366]:
op_file_path = "./Data/ing_desnsity_2.csv"
ing_dens_comb.to_csv(op_file_path, index=False)

In [ ]:
api_key = "6QckmmkUv58RO4eu9eiXKUD1a0L60mSjTnHfa036"
def search_usda_food_with_fdcid(fdcid):
    '''
    given fdc id, return the detailed info
    '''
    url = "https://api.nal.usda.gov/fdc/v1/food/{}?api_key={}".format(fdcid, api_key)
    response = requests.request("GET", url)

    if response.status_code != 200:
       return ''
    
    return response.json()
def search_fdcid(page):
    '''
    Return the fdc id of the search for the first one with food portions
    '''
    url = "https://api.nal.usda.gov/fdc/v1/foods/search?query=&pageNumber={}&pageSize=40&model=foodPortion&api_key=6QckmmkUv58RO4eu9eiXKUD1a0L60mSjTnHfa036"\
          .format(page)
    response = requests.request("GET", url)

#     if response.status_code != 200:
#         return None
    fdc_id_list = []
    names=[]
    for food in response.json().get("foods"):

        fdc_id=food.get("fdcId")
        json_data = search_usda_food_with_fdcid(fdc_id)
        if isinstance(json_data, str):
            continue
#         print(search_usda_food_with_fdcid(fdc_id).get("foodPortions"))
        if( json_data.get("foodPortions") !=None ):
            if (len(json_data.get("foodPortions")) > 0):
                fdc_id_list.append(food.get("fdcId"))
                names.append(food.get("lowercaseDescription"))
    return fdc_id_list, names


In [ ]:
import time

ingredient_density={}
start = time.time()
ing_name=[]
standard_vol=[]
standard_unit=[]
standard_weight=[]
# for index, row in combined_ingredient_df.iterrows():
#     print(row.ingredient)
for page in range(201,210):
    time.sleep(5.0)

    fdcid_list , names = search_fdcid(page)
    for name, fdcid in zip(names, fdcid_list):
        json_data = search_usda_food_with_fdcid(fdcid)
        
        if isinstance(json_data, str):
            continue
        elif 'portionDescription' in json_data.get("foodPortions")[0].keys():
            ing_amt = json_data.get("foodPortions")[0]['portionDescription'].split(' ')[0]
            if ing_amt.isdigit():
                standard_vol.append(int(ing_amt))
                standard_unit.append(json_data.get("foodPortions")[0]['portionDescription'].split(' ')[1])
                standard_weight.append(json_data.get("foodPortions")[0]['gramWeight'])
                ing_name.append(name)                  
        elif ('amount' in json_data.get("foodPortions")[0].keys()) and ('modifier' in json_data.get("foodPortions")[0].keys()):
            standard_vol.append(json_data.get("foodPortions")[0]["amount"])
            standard_unit.append(json_data.get("foodPortions")[0]["modifier"])
            standard_weight.append(json_data.get("foodPortions")[0]["gramWeight"])
            ing_name.append(name)                        
        else:
            standard_vol.append(np.nan)
            standard_weight.append(np.nan)
            standard_unit.append('')
            ing_name.append(name)                         

           
# ing_name = [item for sublist in ing_name for item in sublist]
ingredient_density['ingredient']= ing_name      
ingredient_density['standard_vol']=standard_vol
ingredient_density['standard_weight_gm'] = standard_weight
ingredient_density['standard_unit'] =standard_unit
ingredient_density['ingredient']
ingredient_density_df = pd.DataFrame.from_dict(ingredient_density)

print("time in minutes {}".format((time.time() - start)/60))